# Play with data

In [1]:
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
from processing import *

In [2]:
# https://github.com/microsoft/WSL/issues/4619
# in mongodb.cfg:   bindIp: 0.0.0.0, 127.0.0.1
client = MongoClient('mongodb://192.168.0.104:27017')
client

MongoClient(host=['192.168.0.104:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
db = client.geometry2
db

Database(MongoClient(host=['192.168.0.104:27017'], document_class=dict, tz_aware=False, connect=True), 'geometry2')

In [4]:
db.files.count_documents({})

ServerSelectionTimeoutError: 192.168.0.104:27017: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 60e414d722795ed8037f3f36, topology_type: Single, servers: [<ServerDescription ('192.168.0.104', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('192.168.0.104:27017: timed out')>]>

In [ ]:
objects = db.objects

In [ ]:
data = pd.DataFrame(list(objects.find().limit(20000)))

In [ ]:
data.head()

In [ ]:
data.groupby(['ClassName']).count().sort_values(['_id'], ascending = False)

In [ ]:
ids  = db.files.find({'Scanned':True}).distinct('FileId')
print(len(ids))
query = {
    'ClassName':
    {
        '$in':[
            'Line',
            'Arc',
            'Text',
            'Polyline',
            'AlignedDimension'
        ]
    },
    'FileId':{
        '$in':ids
    }
}

dd = pd.DataFrame(list(db.objects.find(query)))
dd

In [ ]:
dd.groupby(['ClassName']).count().sort_values(['_id'], ascending = False)

# Visual check of drawings

In [ ]:
# links
# https://arxiv.org/pdf/1704.03477.pdf
# https://arxiv.org/pdf/1904.02632.pdf

In [ ]:
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
# to reload .py files
%load_ext autoreload
%autoreload 2

In [ ]:
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas
import pymongo
import pandas as pd
import numpy as np

from pymongo import MongoClient

from IPython.display import Image

# !pip install drawsvg
# Works on linux only!
# sudo apt-get install libcairo2
import drawSvg as draw

In [ ]:
#test drawSvg
# https://pypi.org/project/drawSvg/

def plotLine(StartPoint, EndPoint):
    d = draw.Drawing(200,100,origin = 'center', displayInline = False)
    d.append(draw.Lines(StartPoint.X, StartPoint.Y, EndPoint.X, EndPoint.Y, close = False, fill='#eeee00', stroke = 'black'))
    d.setPixelScale(2)
    d.rasterize()
    return(d)

class pnt2:
    def __init__(self, x,y):
        self.X = x
        self.Y = y
start = pnt2(100,200)
end = pnt2(0,0)
plotLine(start,end)

In [ ]:
client = MongoClient('mongodb://192.168.0.104:27017/')
#client = MongoClient('mongodb://localhost:27017')
client

In [ ]:
db = client.geometry2
db

In [ ]:
objects = db.objects
objects.count()

In [ ]:
fileidsWithDims = objects.find({'ClassName':'AlignedDimension'}).distinct('GroupId')
len(fileidsWithDims)

In [ ]:
from processing import query_collection_to_dataframe, normalize, expand_columns
# f_id = fileidsWithDims[1]
f_id = '1317d221-8d9e-4e2e-b290-3be2a0aa67fb'
debug_data = query_collection_to_dataframe(objects, f_id)    
cols_to_expand = ['XLine1Point', 'XLine2Point', 'StartPoint', 'EndPoint', 'Position']
debug_data = expand_columns(debug_data, cols_to_expand)
normalize(debug_data).describe()

In [ ]:
from plot_graphics import generate_image_by_id
for f_id in fileidsWithDims:
    filename, d = generate_image_by_id(objects, f_id)
    display(d)